# 第22章：LangSmith Integration（LangSmith集成）

## 学习目标

本章将学习：
1. LangSmith简介与核心功能
2. 配置和启用Tracing（追踪）
3. 查看和分析Traces
4. 性能监控与Dashboard
5. Prompt管理和版本控制
6. 数据集管理
7. 在线评估
8. 实战：完整的LangSmith工作流

---

## 什么是LangSmith？

### 核心定位

**LangSmith** 是LangChain官方提供的**生产级LLM应用开发平台**，提供：

| 功能模块 | 说明 | 价值 |
|---------|------|------|
| **Observability** | 追踪、监控、调试 | 可视化执行流程 |
| **Evaluation** | 离线/在线评估 | 确保应用质量 |
| **Prompt Engineering** | Prompt管理和测试 | 协作优化提示词 |
| **Deployment** | Agent部署 | 生产环境运行 |

### 核心概念

```
Trace（追踪）
  ↓
Run（运行）- 单个操作（LLM调用、工具调用等）
  ↓
Project（项目）- 组织Traces的容器
  ↓
Dataset（数据集）- 评估用的测试数据
  ↓
Experiment（实验）- 在Dataset上运行的评估
```

### 为什么需要LangSmith？

✅ **调试困难** → Tracing可视化每一步  
✅ **性能未知** → 监控延迟、成本、错误率  
✅ **质量难保证** → 自动化评估和测试  
✅ **Prompt难管理** → 版本控制和协作  
✅ **生产部署复杂** → 一站式部署方案

---

## 核心架构

### LangSmith vs LangChain

| 维度 | LangChain | LangSmith |
|------|-----------|-----------|
| **定位** | 开源框架 | 商业平台 |
| **功能** | 构建应用 | 监控、评估、部署 |
| **使用方式** | 本地代码 | 云端服务 |
| **关系** | 可独立使用 | 与LangChain无缝集成 |

**关键**：LangSmith是框架无关的，可以与任何LLM应用集成！


In [3]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI

# 配置LangSmith（实际使用）
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = config.LANGSMITH_API_KEY
os.environ["LANGSMITH_PROJECT"] = "langchain-learning"  # 项目名称
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"

# 初始化模型
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=config.CLOUD_API_KEY,
    base_url=config.CLOUD_BASE_URL,
)

print("环境配置完成！")
print(f"模型: {model.model_name}")
print(f"LangSmith Project: {os.environ.get('LANGSMITH_PROJECT')}")
print(f"LangSmith Tracing: {os.environ.get('LANGSMITH_TRACING')}")
print(f"LangSmith API Key: {config.LANGSMITH_API_KEY[:20]}..." if config.LANGSMITH_API_KEY else "")
print("\n✓ LangSmith已启用，所有操作会自动追踪！")
print("✓ 访问 https://smith.langchain.com 查看Traces")

环境配置完成！
模型: gpt-4.1-mini
LangSmith Project: langchain-learning
LangSmith Tracing: true
LangSmith API Key: lsv2_pt_492f9679b1e2...

✓ LangSmith已启用，所有操作会自动追踪！
✓ 访问 https://smith.langchain.com 查看Traces


## 1. 配置LangSmith Tracing

### 启用Tracing的方式

LangSmith提供多种方式启用Tracing：

#### 方式1：环境变量（推荐）

```bash
export LANGSMITH_TRACING=true
export LANGSMITH_API_KEY="your-api-key"
export LANGSMITH_PROJECT="my-project"  # 可选，默认为"default"
```

#### 方式2：代码配置

```python
import os
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "your-api-key"
os.environ["LANGSMITH_PROJECT"] = "my-project"
```

#### 方式3：使用tracing_context（细粒度控制）

```python
from langchain.globals import set_tracing_enabled

# 全局启用
set_tracing_enabled(True)

# 或者使用上下文管理器
with tracing_context(enabled=True, project_name="my-project"):
    # 这里的代码会被追踪
    pass
```

### 配置项说明

| 环境变量 | 说明 | 默认值 |
|---------|------|--------|
| `LANGSMITH_TRACING` | 是否启用追踪 | `false` |
| `LANGSMITH_API_KEY` | API密钥 | 无 |
| `LANGSMITH_PROJECT` | 项目名称 | `"default"` |
| `LANGSMITH_ENDPOINT` | API端点 | `https://api.smith.langchain.com` |

### 自动集成

**关键**：LangChain的`create_agent`自动支持LangSmith追踪，无需额外代码！


In [5]:
print("【演示1：测试LangSmith Tracing】\n")

# 实际测试：发送一个简单的LLM调用到LangSmith
print("=== 测试1：简单的LLM调用 ===\n")

# 这个调用会自动被追踪到LangSmith
response = model.invoke("用一句话介绍Python")
print(f"回答: {response.content}")
print("✓ 这次调用已自动记录到LangSmith")
print()

# 测试2：添加自定义元数据
print("=== 测试2：带元数据的调用 ===\n")

from langchain_core.runnables import RunnableConfig

response2 = model.invoke(
    "什么是LangChain？",
    config=RunnableConfig(
        tags=["demo", "testing"],
        metadata={
            "user_id": "demo_user",
            "environment": "learning",
            "test_case": "langsmith_demo"
        }
    )
)
print(f"回答: {response2.content[:50]}...")
print("✓ 这次调用带有自定义标签和元数据")
print()

print("=== 查看Traces ===\n")
print("现在访问 https://smith.langchain.com")
print(f"1. 选择项目: {os.environ.get('LANGSMITH_PROJECT')}")
print("2. 你会看到刚才的2次LLM调用")
print("3. 点击Trace查看详细信息：")
print("   - 输入/输出")
print("   - 延迟")
print("   - Token使用")
print("   - 自定义元数据和标签")
print()

print("💡 关键收获：")
print("  ✓ 配置环境变量后，所有操作自动追踪")
print("  ✓ 可以添加自定义元数据便于过滤和分析")
print("  ✓ LangSmith UI提供可视化的执行流程")

【演示1：测试LangSmith Tracing】

=== 测试1：简单的LLM调用 ===

回答: Python是一种简洁易学、功能强大的高级编程语言，广泛应用于数据分析、人工智能、Web开发等领域。
✓ 这次调用已自动记录到LangSmith

=== 测试2：带元数据的调用 ===

回答: LangChain 是一个用于构建基于大型语言模型（LLM）的应用程序的开源框架。它提供了一套工具和...
✓ 这次调用带有自定义标签和元数据

=== 查看Traces ===

现在访问 https://smith.langchain.com
1. 选择项目: langchain-learning
2. 你会看到刚才的2次LLM调用
3. 点击Trace查看详细信息：
   - 输入/输出
   - 延迟
   - Token使用
   - 自定义元数据和标签

💡 关键收获：
  ✓ 配置环境变量后，所有操作自动追踪
  ✓ 可以添加自定义元数据便于过滤和分析
  ✓ LangSmith UI提供可视化的执行流程


## 2. Trace结构与查看

### Trace的层次结构

```
Trace（完整请求）
  ├─ Run 1: Agent决策
  │   ├─ Run 1.1: LLM调用
  │   └─ Run 1.2: 工具调用
  │       └─ Run 1.2.1: 工具执行
  ├─ Run 2: Agent决策
  │   └─ Run 2.1: LLM调用（最终答案）
  └─ ...
```

### Run的类型

| Run类型 | 说明 | 示例 |
|--------|------|------|
| **Chain** | 链式调用 | Agent执行流程 |
| **LLM** | 大模型调用 | ChatOpenAI.invoke() |
| **Tool** | 工具调用 | search_web() |
| **Retriever** | 检索器 | vector_store.similarity_search() |

### Trace包含的信息

- **输入/输出**：每个Run的输入和输出
- **延迟**：执行时间
- **Token使用**：输入/输出token数
- **成本**：估算的API成本
- **元数据**：自定义标签和元数据
- **错误信息**：如果发生错误


In [6]:
print("【演示2：Agent追踪】\n")

# 创建一个带工具的Agent，查看完整的执行流程
from langchain.agents import create_agent
from langchain.tools import tool

# 定义一个简单的工具
@tool
def get_weather(city: str) -> str:
    """获取指定城市的天气信息"""
    # 模拟天气查询
    weather_data = {
        "北京": "晴天，25°C",
        "上海": "多云，22°C",
        "深圳": "小雨，28°C"
    }
    return weather_data.get(city, f"{city}的天气数据暂时不可用")

# 创建Agent
print("=== 创建Agent ===\n")
agent = create_agent(
    model=model,
    tools=[get_weather],
    system_prompt="你是一个天气助手，可以帮用户查询天气信息。"
)
print("✓ Agent创建完成，包含1个工具：get_weather")
print()

# 测试1：简单查询
print("=== 测试1：简单查询（自动追踪） ===\n")
result1 = agent.invoke({
    "messages": [{"role": "user", "content": "北京今天天气怎么样？"}]
})
print(f"回答: {result1['messages'][-1].content}")
print("✓ 这次Agent调用已记录到LangSmith（包含工具调用）")
print()

# 测试2：带元数据
print("=== 测试2：带元数据和标签 ===\n")
result2 = agent.invoke(
    {"messages": [{"role": "user", "content": "上海和深圳哪个城市天气更好？"}]},
    config=RunnableConfig(
        tags=["weather", "comparison"],
        metadata={
            "user_id": "user_123",
            "session_id": "session_abc",
            "query_type": "comparison"
        }
    )
)
print(f"回答: {result2['messages'][-1].content}")
print("✓ 这次调用带有自定义标签和元数据")
print()

print("=== 在LangSmith查看 ===\n")
print("访问 https://smith.langchain.com 你会看到：")
print("1. Agent的完整执行轨迹：")
print("   - 用户输入")
print("   - Agent决策（调用哪个工具）")
print("   - 工具执行")
print("   - Agent生成最终答案")
print("2. 每个步骤的延迟和Token使用")
print("3. 自定义的标签和元数据（可用于过滤）")
print()

print("💡 关键收获：")
print("  ✓ Agent的每个步骤都被自动追踪")
print("  ✓ 可以看到工具调用的完整过程")
print("  ✓ 元数据帮助组织和过滤大量Traces")


【演示2：Agent追踪】

=== 创建Agent ===

✓ Agent创建完成，包含1个工具：get_weather

=== 测试1：简单查询（自动追踪） ===

回答: 北京今天天气晴朗，气温约为25°C。需要我帮您查询其他城市的天气吗？
✓ 这次Agent调用已记录到LangSmith（包含工具调用）

=== 测试2：带元数据和标签 ===

回答: 上海的天气是多云，温度22度；深圳的天气是小雨，温度28度。根据天气情况来看，上海的天气更好一些，因为没有下雨。您更喜欢哪种天气呢？
✓ 这次调用带有自定义标签和元数据

=== 在LangSmith查看 ===

访问 https://smith.langchain.com 你会看到：
1. Agent的完整执行轨迹：
   - 用户输入
   - Agent决策（调用哪个工具）
   - 工具执行
   - Agent生成最终答案
2. 每个步骤的延迟和Token使用
3. 自定义的标签和元数据（可用于过滤）

💡 关键收获：
  ✓ Agent的每个步骤都被自动追踪
  ✓ 可以看到工具调用的完整过程
  ✓ 元数据帮助组织和过滤大量Traces


## 3. Prompt管理与版本控制

### 核心价值

LangSmith提供了**Prompt Hub**，用于：
- 版本控制Prompt
- 团队协作编辑
- 在代码中动态拉取Prompt
- A/B测试不同版本

### Prompt管理工作流

```
1. 在LangSmith UI中创建/编辑Prompt
   ↓
2. 提交（Commit）新版本
   ↓
3. 在代码中拉取Prompt
   ↓
4. 使用Prompt
   ↓
5. 根据反馈迭代
```

### Prompt版本控制

| 概念 | 说明 | 示例 |
|------|------|------|
| **Commit** | 每次保存创建新版本 | v1, v2, v3... |
| **Tag** | 标记重要版本 | `prod`, `staging` |
| **Latest** | 最新版本 | 自动更新 |

### 使用Prompt的方式

```python
from langsmith import Client

client = Client()

# 方式1：拉取最新版本
prompt = client.pull_prompt("my-prompt")

# 方式2：拉取特定版本
prompt = client.pull_prompt("my-prompt:v2")

# 方式3：拉取带标签的版本
prompt = client.pull_prompt("my-prompt:prod")

# 使用Prompt
formatted = prompt.format(question="What is Python?")
```


In [7]:
print("【演示3：Prompt管理】\n")

# 实际演示Prompt的创建、推送和使用
from langsmith import Client
from langchain_core.prompts import ChatPromptTemplate

# 创建LangSmith客户端
client = Client(api_key=config.LANGSMITH_API_KEY)

print("=== 步骤1：创建Prompt模板 ===\n")
# 创建一个简单的QA Prompt
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个知识渊博的助手，擅长用简洁的语言回答问题。"),
    ("human", "{question}")
])
print("✓ 创建了一个QA Prompt模板")
print()

print("=== 步骤2：推送Prompt到LangSmith ===\n")
try:
    # 推送Prompt（会创建新版本）
    prompt_url = client.push_prompt(
        "learning-qa-prompt",  # Prompt名称
        object=qa_prompt
    )
    print(f"✓ Prompt已推送到LangSmith")
    print(f"  URL: {prompt_url}")
except Exception as e:
    print(f"注意: {e}")
    print("  (如果Prompt已存在，会创建新版本)")
print()

print("=== 步骤3：拉取并使用Prompt ===\n")
try:
    # 从LangSmith拉取Prompt
    pulled_prompt = client.pull_prompt("learning-qa-prompt")
    print("✓ 成功从LangSmith拉取Prompt")
    
    # 使用拉取的Prompt
    formatted = pulled_prompt.format(question="什么是机器学习？")
    response = model.invoke(formatted)
    print(f"\n问题: 什么是机器学习？")
    print(f"回答: {response.content[:100]}...")
    print("\n✓ 使用从LangSmith拉取的Prompt成功生成回答")
except Exception as e:
    print(f"拉取失败: {e}")
    print("  可能原因：Prompt名称不存在或需要等待同步")
print()

print("=== 查看Prompt管理 ===\n")
print("访问 https://smith.langchain.com")
print("1. 进入 'Prompts' 页面")
print("2. 找到 'learning-qa-prompt'")
print("3. 查看版本历史")
print("4. 可以在UI中编辑、添加Tag等")
print()

print("💡 Prompt管理的价值：")
print("  ✓ 版本控制：每次修改都有记录")
print("  ✓ 团队协作：多人可以在UI中编辑")
print("  ✓ 动态更新：修改Prompt不需要重新部署代码")
print("  ✓ 生产环境可以用Tag（如:prod）固定版本")

【演示3：Prompt管理】

=== 步骤1：创建Prompt模板 ===

✓ 创建了一个QA Prompt模板

=== 步骤2：推送Prompt到LangSmith ===

✓ Prompt已推送到LangSmith
  URL: https://smith.langchain.com/prompts/learning-qa-prompt/81c7ffc0?organizationId=fb57ca02-34e6-49b5-ba81-59720cd86f4e

=== 步骤3：拉取并使用Prompt ===

✓ 成功从LangSmith拉取Prompt

问题: 什么是机器学习？
回答: 机器学习是一种让计算机通过数据自动学习和改进的技术，而无需明确编程。它通过算法分析数据，发现规律，从而进行预测或决策。...

✓ 使用从LangSmith拉取的Prompt成功生成回答

=== 查看Prompt管理 ===

访问 https://smith.langchain.com
1. 进入 'Prompts' 页面
2. 找到 'learning-qa-prompt'
3. 查看版本历史
4. 可以在UI中编辑、添加Tag等

💡 Prompt管理的价值：
  ✓ 版本控制：每次修改都有记录
  ✓ 团队协作：多人可以在UI中编辑
  ✓ 动态更新：修改Prompt不需要重新部署代码
  ✓ 生产环境可以用Tag（如:prod）固定版本


## 4. 数据集管理与评估

### 数据集（Dataset）

LangSmith的Dataset用于评估，包含：
- **Examples**：测试样本（输入+预期输出）
- **Splits**：数据集分割（train/test）
- **Versions**：数据集版本

### 创建Dataset的方式

| 方式 | 说明 | 适用场景 |
|------|------|---------|
| **UI创建** | 在LangSmith UI手动创建 | 小规模数据集 |
| **SDK创建** | 通过代码批量创建 | 大规模数据集 |
| **从Traces导出** | 从生产Traces创建 | 真实场景测试 |

### 评估工作流

```
1. 创建Dataset
   ↓
2. 定义评估函数（target）
   ↓
3. 定义评估器（evaluators）
   ↓
4. 运行evaluate()
   ↓
5. 在LangSmith UI查看结果
```

### 离线评估 vs 在线评估

| 类型 | 时机 | 数据源 | 用途 |
|------|------|--------|------|
| **离线评估** | 部署前 | 测试数据集 | 对比版本、回归测试 |
| **在线评估** | 生产中 | 真实流量 | 监控质量、发现问题 |


In [8]:
print("【演示4：数据集管理与评估】\n")

# 实际创建数据集并运行评估
from langsmith import evaluate
from langsmith.schemas import Example, Run

print("=== 步骤1：创建Dataset ===\n")
dataset_name = "learning-qa-dataset"

try:
    # 尝试创建数据集（如果已存在会报错，我们会捕获）
    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description="学习用的QA评估数据集"
    )
    print(f"✓ 创建新数据集: {dataset_name}")
except Exception as e:
    print(f"数据集已存在或创建失败: {e}")
    print("  将使用现有数据集")

# 添加测试样本
examples_data = [
    {
        "inputs": {"question": "什么是Python？"},
        "outputs": {"expected": "Python是一种编程语言"}
    },
    {
        "inputs": {"question": "什么是机器学习？"},
        "outputs": {"expected": "机器学习是人工智能的一个分支"}
    },
    {
        "inputs": {"question": "LangChain有什么用？"},
        "outputs": {"expected": "LangChain用于构建LLM应用"}
    }
]

print(f"\n添加 {len(examples_data)} 个测试样本...\n")
for i, ex in enumerate(examples_data, 1):
    try:
        client.create_example(
            dataset_name=dataset_name,
            inputs=ex["inputs"],
            outputs=ex["outputs"]
        )
        print(f"  ✓ 样本 {i}: {ex['inputs']['question'][:30]}...")
    except Exception as e:
        print(f"  - 样本 {i} 可能已存在")

print(f"\n✓ 数据集 '{dataset_name}' 准备完成")
print()

print("=== 步骤2：定义评估函数 ===\n")

# 要评估的应用
def qa_application(inputs: dict) -> dict:
    """简单的QA应用"""
    question = inputs["question"]
    response = model.invoke(question)
    return {"answer": response.content}

# 评估器：检查长度
def length_evaluator(run: Run, example: Example) -> dict:
    """检查回答长度是否合理"""
    answer = run.outputs.get("answer", "")
    score = 1.0 if 10 < len(answer) < 500 else 0.5
    return {
        "key": "length",
        "score": score,
        "comment": f"长度: {len(answer)} 字符"
    }

print("✓ 定义了评估函数和评估器")
print()

print("=== 步骤3：运行评估 ===\n")
print("开始评估（这需要一些时间）...\n")

try:
    results = evaluate(
        qa_application,
        data=dataset_name,
        evaluators=[length_evaluator],
        experiment_prefix="learning-qa",
        max_concurrency=2
    )
    
    print("✓ 评估完成！")
    print(f"\n实验名称: {results.experiment_name}")
    print(f"数据集: {dataset_name}")
    print(f"样本数: {len(examples_data)}")
    print()
    
except Exception as e:
    print(f"评估失败: {e}")
    print("  可能原因：数据集为空或网络问题")
print()

print("=== 查看结果 ===\n")
print("访问 https://smith.langchain.com")
print(f"1. 进入 'Datasets' → '{dataset_name}'")
print("2. 查看最新的实验结果")
print("3. 你会看到：")
print("   - 每个样本的实际输出")
print("   - 评估器的分数")
print("   - 延迟和Token统计")
print("   - 可以对比不同实验")
print()

print("💡 评估的价值：")
print("  ✓ 自动化测试：无需手动逐个检查")
print("  ✓ 版本对比：快速发现性能退化")
print("  ✓ 数据驱动：基于客观数据做决策")


【演示4：数据集管理与评估】

=== 步骤1：创建Dataset ===

✓ 创建新数据集: learning-qa-dataset

添加 3 个测试样本...

  ✓ 样本 1: 什么是Python？...
  ✓ 样本 2: 什么是机器学习？...
  ✓ 样本 3: LangChain有什么用？...

✓ 数据集 'learning-qa-dataset' 准备完成

=== 步骤2：定义评估函数 ===

✓ 定义了评估函数和评估器

=== 步骤3：运行评估 ===

开始评估（这需要一些时间）...



/home/iip/miniconda3/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'learning-qa-dc81dc87' at:
https://smith.langchain.com/o/fb57ca02-34e6-49b5-ba81-59720cd86f4e/datasets/65f2fd29-9969-4080-98a3-97ec7ac70c87/compare?selectedSessions=9d5cd831-59f3-4516-8d69-d9e3e77e576d




3it [00:10,  3.55s/it]

✓ 评估完成！

实验名称: learning-qa-dc81dc87
数据集: learning-qa-dataset
样本数: 3


=== 查看结果 ===

访问 https://smith.langchain.com
1. 进入 'Datasets' → 'learning-qa-dataset'
2. 查看最新的实验结果
3. 你会看到：
   - 每个样本的实际输出
   - 评估器的分数
   - 延迟和Token统计
   - 可以对比不同实验

💡 评估的价值：
  ✓ 自动化测试：无需手动逐个检查
  ✓ 版本对比：快速发现性能退化
  ✓ 数据驱动：基于客观数据做决策


## 5. 性能监控与Dashboard

### 监控指标

LangSmith自动收集以下指标：

| 指标类别 | 具体指标 | 用途 |
|---------|---------|------|
| **延迟** | P50/P95/P99延迟 | 性能优化 |
| **成本** | Token使用、API成本 | 成本控制 |
| **质量** | 错误率、评估分数 | 质量监控 |
| **使用量** | 请求数、用户数 | 容量规划 |

### Dashboard功能

- **时间序列图表**：查看指标随时间变化
- **过滤和分组**：按标签、环境、版本分析
- **告警规则**：设置阈值触发告警
- **对比分析**：对比不同版本性能

### 监控最佳实践

1. **设置基线**：记录正常情况下的指标范围
2. **分层监控**：
   - 应用层：整体请求成功率、延迟
   - 组件层：LLM调用、工具调用性能
   - 用户层：按用户/会话分析
3. **告警策略**：
   - 错误率 > 5%
   - P95延迟 > 3秒
   - 成本异常增长
4. **定期回顾**：每周查看Dashboard，发现趋势


## 6. 完整工作流示例

### 典型的LangSmith工作流

```
开发阶段：
1. 启用Tracing，观察执行流程
2. 在Prompt Hub创建和测试Prompt
3. 创建测试数据集
4. 运行离线评估，对比版本

部署阶段：
5. 标记生产Prompt版本（:prod tag）
6. 配置监控Dashboard
7. 设置告警规则

生产阶段：
8. 持续收集Traces
9. 监控性能指标
10. 从生产Traces补充测试数据集
11. 定期运行评估
12. 迭代优化Prompt和模型
```

### 团队协作

| 角色 | 使用LangSmith的方式 |
|------|-------------------|
| **开发者** | 查看Traces调试、运行评估 |
| **Prompt工程师** | 在Prompt Hub编辑和测试 |
| **QA** | 管理测试数据集、运行评估 |
| **运维** | 监控Dashboard、处理告警 |
| **产品经理** | 查看质量指标、用户反馈 |


In [9]:
print("【演示5：完整工作流总结】\n")

# 回顾本章学到的完整工作流

print("=== 我们已经完成的步骤 ===\n")
print("✓ 步骤1：配置LangSmith（Cell 1）")
print("  - 设置环境变量")
print("  - 启用自动追踪")
print()

print("✓ 步骤2：测试基本追踪（Cell 3）")
print("  - 简单的LLM调用")
print("  - 带元数据的调用")
print()

print("✓ 步骤3：创建和追踪Agent（Cell 5）")
print("  - 创建带工具的Agent")
print("  - 查看完整的执行流程")
print()

print("✓ 步骤4：Prompt管理（Cell 7）")
print("  - 推送Prompt到LangSmith")
print("  - 从LangSmith拉取Prompt")
print()

print("✓ 步骤5：数据集和评估（Cell 9）")
print("  - 创建测试数据集")
print("  - 运行自动化评估")
print()

print("=== 完整的LangSmith工作流 ===\n")
print("开发阶段：")
print("  1. 启用Tracing → 所有操作自动记录")
print("  2. 在UI中查看Traces → 理解执行流程")
print("  3. 创建Prompt → 在Prompt Hub管理")
print("  4. 创建测试集 → 准备评估数据")
print()

print("测试阶段：")
print("  5. 运行评估 → 自动化测试")
print("  6. 查看结果 → 分析性能")
print("  7. 迭代优化 → 改进Prompt/模型")
print()

print("生产阶段：")
print("  8. 标记Prompt版本（:prod tag）")
print("  9. 持续监控 → Dashboard和告警")
print("  10. 从生产Traces补充测试集")
print()

print("=== 访问LangSmith查看所有结果 ===\n")
print("🌐 https://smith.langchain.com")
print()
print(f"项目: {os.environ.get('LANGSMITH_PROJECT')}")
print()
print("你现在可以看到：")
print("  📊 Traces - 所有的执行记录")
print("  📝 Prompts - learning-qa-prompt")
print("  📁 Datasets - learning-qa-dataset")
print("  🧪 Experiments - 评估结果")
print()

print("💡 关键收获：")
print("  ✓ LangSmith让LLM应用开发变得可观测、可评估、可迭代")
print("  ✓ 配置简单，只需环境变量即可启用")
print("  ✓ 自动集成，LangChain应用无需修改代码")
print("  ✓ 提供完整的开发到生产的支持")
print()
print("🎉 恭喜！你已经掌握了LangSmith的核心功能！")


【演示5：完整工作流总结】

=== 我们已经完成的步骤 ===

✓ 步骤1：配置LangSmith（Cell 1）
  - 设置环境变量
  - 启用自动追踪

✓ 步骤2：测试基本追踪（Cell 3）
  - 简单的LLM调用
  - 带元数据的调用

✓ 步骤3：创建和追踪Agent（Cell 5）
  - 创建带工具的Agent
  - 查看完整的执行流程

✓ 步骤4：Prompt管理（Cell 7）
  - 推送Prompt到LangSmith
  - 从LangSmith拉取Prompt

✓ 步骤5：数据集和评估（Cell 9）
  - 创建测试数据集
  - 运行自动化评估

=== 完整的LangSmith工作流 ===

开发阶段：
  1. 启用Tracing → 所有操作自动记录
  2. 在UI中查看Traces → 理解执行流程
  3. 创建Prompt → 在Prompt Hub管理
  4. 创建测试集 → 准备评估数据

测试阶段：
  5. 运行评估 → 自动化测试
  6. 查看结果 → 分析性能
  7. 迭代优化 → 改进Prompt/模型

生产阶段：
  8. 标记Prompt版本（:prod tag）
  9. 持续监控 → Dashboard和告警
  10. 从生产Traces补充测试集

=== 访问LangSmith查看所有结果 ===

🌐 https://smith.langchain.com

项目: langchain-learning

你现在可以看到：
  📊 Traces - 所有的执行记录
  📝 Prompts - learning-qa-prompt
  📁 Datasets - learning-qa-dataset
  🧪 Experiments - 评估结果

💡 关键收获：
  ✓ LangSmith让LLM应用开发变得可观测、可评估、可迭代
  ✓ 配置简单，只需环境变量即可启用
  ✓ 自动集成，LangChain应用无需修改代码
  ✓ 提供完整的开发到生产的支持

🎉 恭喜！你已经掌握了LangSmith的核心功能！


## 7. 总结与关键收获

### 核心要点

#### 1. LangSmith的四大核心功能

| 功能 | 价值 | 关键特性 |
|------|------|---------|
| **Observability** | 调试和理解 | Tracing、元数据、过滤 |
| **Evaluation** | 确保质量 | 数据集、评估器、实验对比 |
| **Prompt Engineering** | 协作优化 | 版本控制、Tag、动态拉取 |
| **Monitoring** | 生产监控 | Dashboard、告警、指标 |

#### 2. Tracing配置

```python
# 最简单的方式：环境变量
export LANGSMITH_TRACING=true
export LANGSMITH_API_KEY="your-key"
export LANGSMITH_PROJECT="my-project"

# LangChain Agent自动支持，无需额外代码！
```

#### 3. 评估工作流

```python
from langsmith import Client, evaluate

# 1. 创建数据集
client = Client()
dataset = client.create_dataset("my-dataset")

# 2. 运行评估
results = evaluate(
    my_app,
    data="my-dataset",
    evaluators=[evaluator1, evaluator2]
)

# 3. 在UI查看结果
```

#### 4. Prompt管理

```python
from langsmith import Client

client = Client()

# 推送Prompt
client.push_prompt("my-prompt", prompt_object)

# 拉取Prompt（生产环境用tag）
prompt = client.pull_prompt("my-prompt:prod")
```

---

### 最佳实践

#### ✅ 推荐做法

1. **开发阶段**
   - 始终启用Tracing
   - 频繁查看Traces理解执行流程
   - 使用Prompt Hub快速迭代

2. **测试阶段**
   - 创建全面的测试数据集
   - 使用多个评估器
   - 对比不同版本

3. **生产阶段**
   - 使用tag管理Prompt版本（:prod）
   - 配置监控Dashboard
   - 设置告警规则
   - 定期从生产Traces补充测试集

4. **团队协作**
   - 统一Project命名规范
   - 使用元数据标记环境和版本
   - 共享数据集和评估器
   - 定期回顾Dashboard

#### ❌ 避免的陷阱

1. **不添加元数据**
   - ❌ 所有Traces混在一起
   - ✅ 添加user_id、environment等元数据

2. **生产环境用:latest**
   - ❌ Prompt变化导致意外行为
   - ✅ 使用:prod等稳定tag

3. **忽略成本监控**
   - ❌ Token使用失控
   - ✅ 定期查看成本Dashboard

4. **测试数据集过时**
   - ❌ 评估结果不反映真实情况
   - ✅ 定期从生产更新数据集

5. **只在出问题时查看**
   - ❌ 问题发现太晚
   - ✅ 主动监控，预防问题

---

### LangSmith的价值

#### 对比传统方式

| 维度 | 传统方式 | 使用LangSmith |
|------|---------|--------------|
| **调试** | print日志 | 可视化Trace |
| **评估** | 手动测试 | 自动化评估 |
| **Prompt管理** | 硬编码 | 版本控制+动态拉取 |
| **监控** | 自建系统 | 开箱即用Dashboard |
| **协作** | 难以共享 | 团队共享平台 |

#### 投资回报

- **开发效率** ↑ 50%：快速定位问题
- **质量保证** ↑ 80%：自动化评估
- **运维成本** ↓ 60%：统一监控平台
- **迭代速度** ↑ 3x：Prompt快速迭代

---

## 下一步学习

完成LangSmith集成后，建议：

1. **实战项目** - 将LangSmith集成到实际项目
2. **高级功能** - 探索在线评估、自动化规则
3. **团队协作** - 建立团队的LangSmith使用规范

---

## 关键收获

✅ **LangSmith是生产级LLM应用的必备工具** - 提供完整的可观测性

✅ **Tracing自动集成** - LangChain Agent无需额外代码

✅ **评估驱动迭代** - 数据驱动的持续优化

✅ **Prompt版本控制** - 团队协作和动态更新

✅ **生产监控** - Dashboard和告警确保稳定性

✅ **框架无关** - 可以与任何LLM应用集成

✅ **投资回报高** - 显著提升开发效率和应用质量

✅ **免费开始** - 提供免费tier，易于上手
